<a href="https://colab.research.google.com/github/100495802/G11.AA-495802-495702/blob/main/P1AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P1-AA: PREDICCIÓN DEL ABANDONO DE EMPLEADOS DE UNA EMPRESA
- Curso de Aprendizaje Automático 2024/25
- Grado en Ingeniería Informática - UC3M
- Alejandro López Sancho: 100495702
- Javier Rosales Lozano: 100495802

## 1. Introducción y carga de datos

### 1.1. Enunciado del proyecto

El siguiente notebook contiene toda la información y trabajo realizado acerca de la primera parte de la práctica inicial del curso de Aprendizaje Automático 2024/25. El objetivo de esta práctica es realizar la __construcción, medición y evaluación de un modelo que prediga un problema planteado como el número de abandonos dentro de una empresa en función de las características de sus trabajadores__.

En este primer archivo de Jupyter Notebook encontraremos todo el proceso de EDA (Exploratory Data Analysis), selección de imputers y scalers, ajuste de hiperparámetros, evaluación de modelos, obtención de la tasa de aprendizaje y resolución de las preguntas establecidas en el enunciado de la práctica.

### 1.2. Repositorio Github y Dataset Inicial

El siguiente enlace redirige al [repositorio Github](https://github.com/100495802/G11.AA-495802-495702.git) correspondiente al proyecto; los datos elegidos para la realización del proyecto corresponden con el estándar especificado para la realización del trabajo (suma de los dos últimos dígitos de uno de los NIAs del grupo de prácticas). En este caso, hemos elegido el NIA __100495702__, por lo que nuestro [dataset inicial](attrition_availabledata_02.csv.gz) corresponderá con el número _2_ del conjunto de datasets del Aula Global.

### 1.3. Carga de Datos

A continuación, nos disponemos a la __carga de este conjunto de datos__:

In [38]:
import pandas as pd
datos = pd.read_csv("attrition_availabledata_02.csv.gz", compression="gzip", sep=",")

# 2. EDA Simplificado

Una vez cargado el dataset, nos disponemos a comentar y describir los distintos datos que existen en el dataset.

### 2.1. Análisis general del dataset

Vamos a investigar primeramente el tamaño del dataset; esto lo haremos con los atributos de la librería Pandas _.shape_ y _.size_. 

In [39]:
print(f"Tamaño del dataset: {datos.shape}")
print(f"Número total de elementos del dataset: {datos.size}")

Tamaño del dataset: (2940, 31)
Número total de elementos del dataset: 91140


Podemos observar que el tamaño del dataset es de __2940 filas (instancias) y 31 columnas (atributos)__. También identificamos el número de elementos del conjunto de datos.

Seguidamente, echaremos un vistazo rápido a los atributos que hay en la base de datos; usaremos el método _info()_ para identificar las diferentes columnas del dataset, además del tipo de dato y el número de instancias no nulas que contiene.

In [40]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2927 non-null   float64
 5   JobSatisfaction          2925 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount            2940 non-null   int64  
 14  EmployeeID              

En la salida podemos observar una tabla donde apreciamos el nombre del atirbuto, el número de instancias no vacías y el tipo de dato de ésta columna. Esto nos servirá para apreciar los tipos de datos que estamos analizando.

Podemos observar que algunos de ellos cumplen con todas sus filas con algún valor no nulo (N/A), mientras que __otras columnas no alcanzan el número total de filas obtenido anteriormente (2940)__.

Por otro lado, el tipo de datos nos hace preveer cuáles columnas representan __atributos categóricos__ (_object_) y cuáles __atributos numéricos__ (_int64, float64_).

Para un resumen más detallado, nos serviremos de otro método de Pandas: _.describe()_

In [41]:
datos.describe()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,2940.000000,2940.000000,2940.000000,2940.000000,2927.000000,2925.000000,2911.000000,2940.000000,2940.000000,2940.000000,...,2940.000000,2926.000000,2940.000000,2940.0,2940.000000,2935.000000,2940.000000,2940.000000,2940.000000,2940.000000
mean,7.322768,12.706803,2.728571,3.154422,2.723266,2.746325,2.760907,36.861224,9.305102,2.906463,...,64379.826531,2.664388,15.187075,8.0,0.785034,11.363884,2.798639,7.095578,2.227891,4.191156
std,1.335600,5.533199,0.716167,0.361414,1.096170,1.104612,0.713539,9.286733,8.201638,1.023254,...,47021.197990,2.495153,3.661275,0.0,0.851343,7.897032,1.304166,6.161878,3.274101,3.627734
min,5.416880,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,...,10090.000000,0.000000,11.000000,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.272786,8.000000,2.000000,3.000000,2.000000,2.000000,2.000000,30.000000,2.000000,2.000000,...,28750.000000,1.000000,12.000000,8.0,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000
50%,7.032627,13.000000,3.000000,3.000000,3.000000,3.000000,3.000000,35.000000,7.000000,3.000000,...,48340.000000,2.000000,14.000000,8.0,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,7.948416,17.000000,3.000000,3.000000,4.000000,4.000000,3.000000,43.000000,14.000000,4.000000,...,80080.000000,4.000000,18.000000,8.0,1.000000,16.000000,3.000000,10.000000,3.000000,7.000000
max,10.937261,24.000000,4.000000,4.000000,4.000000,4.000000,4.000000,60.000000,29.000000,5.000000,...,199990.000000,9.000000,25.000000,8.0,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


Ésta ejecución de código nos permite identificar algunas __métricas estadísticas__ acerca del conjunto de datos que abarcan los distintos atributos (como la media, la mediana, mínimos, máximos y rangos intercuartílicos). Esto de momento no nos servirá, pero quizás más adelante será útil.

Con los datos visualizados anteriormente en la ejecución de _.info()_, vamos a observar los valores de las variables más a fondo, haciendo hincapié en las distintas observaciones que se piden en la práctica.

### 2.2. Variables numéricas y categóricas; variables con alta cardinalidad

Empezamos distinguiendo los atributos que representan __valores categóricos__ de los que representan __valores numéricos__ (descartando la columna de la variable de salida, _Attrition_, la cual sabemos que es categórica):

In [42]:
numericas = datos.select_dtypes(include=['int64', 'float64']).columns
print(numericas)

Index(['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeID',
       'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [43]:
categoricas = datos.select_dtypes(include=["object"]).columns.drop("Attrition")
print(categoricas)

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'Over18'],
      dtype='object')


A continuación, de las __variables categóricas__, vamos a identificar aquellas variables con una __alta cardinalidad__ en sus valores:

In [44]:
for columna in datos.select_dtypes(include=["object"]).columns:
    print(columna)
    print(datos[columna].unique())

BusinessTravel
['Travel_Frequently' 'Non-Travel' 'Travel_Rarely']
Department
['Research & Development' 'Sales' 'Human Resources']
EducationField
['Life Sciences' 'Medical' 'Other' 'Technical Degree' 'Marketing'
 'Human Resources']
Gender
['Male' 'Female']
JobRole
['Laboratory Technician' 'Healthcare Representative' 'Research Scientist'
 'Sales Representative' 'Manufacturing Director' 'Sales Executive'
 'Research Director' 'Human Resources' 'Manager']
MaritalStatus
['Married' 'Divorced' 'Single']
Over18
['Y']
Attrition
['Yes' 'No']


Analizando la salida de esta ejecución, podemos observar qué variables categóricas presentan elevada cardinalidad. __En este proyecto consideraremos con cardinalidad elevada un atributo con rango de posibles opciones mayor de 4__.

En este caso, las columnas con alta cardinalidad son _EducationField_ y _JobRole_.

### 2.3. Variables nulas

Para el preprocesado de datos, es importante determinar qué instancias contienen __valores nulos__, y el número de instancias de este tipo en cada columna. Esto lo comprobamos con el método de Pandas _.isnull()_:

In [45]:
nulos = datos.isnull().sum()
nulos[nulos > 0]

EnvironmentSatisfaction    13
JobSatisfaction            15
WorkLifeBalance            29
NumCompaniesWorked         14
TotalWorkingYears           5
dtype: int64

Como podemos observar, en el dataset se aprecian __cinco columnas con valores nulos__ en sus filas: _EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance, NumCompaniesWorked, TotalWorkingYears_. Teniendo en cuenta que en total hay 2940 filas en la base de datos, se pueden calcular fácilmente la media de datos vacíos por columna.

Si recordamos de qué tipo era cada variable, podemos observar que __todas estas columnas corresponden con variables numéricas__, y podemos ir pensando ya en posibilidades para trabajar con estos valores vacíos.

### 2.4. Variables de identificación (ID)

Una vez analizado todo lo anterior, empezamos a buscar qué posibles columnas actúan como identificadores en la base de datos. A simple vista, una posible opción sería _EmployeeID_, aunque es mejor comprobarlo con la llamada al método _.nunique()_:

In [46]:
for columna in datos.columns:
    if datos[columna].nunique() == len(datos):
        print(f"La columna '{columna}' es un posible identificador único.")

La columna 'EmployeeID' es un posible identificador único.


In [ ]:
datos.drop("EmployeeNumber", axis=1)

KeyError: "['EmployeeNumber'] not found in axis"

Nuestras sospechas han resultado ser ciertas, y es que es el único atributo cuyos valores son únicos, y no coinciden dos o más instancias con el mismo valor en esta columna. Más adelante decidiremos qué hacer con esta columna.

### 2.5. ¿Problema desbalanceado?

Con el planteamiento completo del enunciado se puede intuir que estamos ante un __problema de clasificación__. Además, la variable _Attrition_ es la que determinará si un empleado es propenso a abandonar la empresa en función del resto de columnas.

Sin embargo, todavía __no podemos comprobar si se trata de un problema de clasificación desbalanceado__, ya que no hemos hecho un conteo de los valores de dicha columna. Este paso es importante, ya que marcará la manera en la que realizaremos las particiones para entrenar, validar y evaluar el modelo.

Para averiguarlo, usaremos el método _.value_counts()_:

In [47]:
datos["Attrition"].value_counts()

Attrition
No     2466
Yes     474
Name: count, dtype: int64

La salida de la ejecución nos muestra el conteo de datos de dicha columna, lo que demuestra que __se trata de un problema desbalanceado__ hacia el valor "no", siendo más del 80% de las clasificaciones de este tipo.

## 3. Metodología de trabajo: Decidir cómo se va a realizar la evaluación

Una vez terminada la parte de análisis del conjunto de datos, podemos concluir en dos cosas:

1. El problema es de clasificación, desbalanceado, con variables categóricas y numéricas, y algunas instancias contienen datos vacíos.

2. Siendo el 80% de la muestra la que decide no abandonar, por el momento, la empresa puede estar tranquila.

Con los datos ya analizados, procedemos a especificar la realización de la práctica. En este apartado vamos a comentar __cómo vamos a evaluar nuestro modelo__.

### 3.1. Planificación del entrenamiento

- __División de los datos:__

  Para realizar la evaluación de nuestro modelo, lo primero que tenemos que hacer es dividir el conjunto de datos en dos particiones de train y test. Usaremos __Holdout__ para invertir 2/3 de los datos para la parte de entrenamiento del modelo y el resto para evaluar el modelo final del entrenamiento.

- __Preprocesado de datos:__

  Para empezar con la fase de entrenamiento, primero deberemos ajustar unas métricas relacionadas con el dataset. Deberemos medir qué método de __imputación__ y __escalado__ de atributos genera un mejor rendimiento de los datos. Para esta medición, usaremos KNN con los hiperparámetros por defecto, y modificaremos el escalado usando varios formatos (_Standard, MinMax, Robust_) y también el tratado de datos vacios o imputación de datos, usando otras métricas (_Media, Mediana_, al tratarse de atributos numéricos). Por último, en todos los casos usaremos la metodología de evaluación de modelos __3-Fold con crossvalidation__. Con todo esto buscamos obtener el Escaler-Imputer que de el mejor rendimiento de la partición de entrenamiento.

- __Creación de Modelos y Ajuste de hyperparametro HPO:__

  Una vez hecho lo anterior, comenzamos la búsqueda del mejor modelo; se construirán una serie de modelos basandonos en las metodologías aprendidas en clase (KNN, Árboles de Decisión, Modelos Lineales y SVMs) mientras realizamos el __ajuste de hiperparámetros__ (HPO), y elegiremos el modelo que mejor se ajuste a los datos, procurando evitar cualquier data-leakage y overfitting/underfitting.

- __Evaluación del modelo final de entrenamiento:__

  Cuando hayamos encontrado la mejor alternativa usaremos el conjunto de test anteriormente apartado para obtener una __estimación y rendimiento a futuro del modelo__. Esto es lo que se conoce como la evaluación del modelo

### 3.2. Entrenamiento y evaluación del modelo final

Los pasos anteriores en conjunto nos dirigen al entrenamiento del modelo final, el cual se entrena con el dataset completo (entrenamiento + test) y evaluaremos realizando predicciones con un conjunto de datos de competición aparte.

Toda esta implementación se realiza en el [segundo Jupyter Notebook](enlacealsiguiente) a entregar con la práctica.

## 4. Métodos Básicos: KNN y TREES; imputación, escalado y ajuste de hiperparámetros

Como hemos establecido anteriormente, el primer paso es realizar el escalado/imputación de los datos, de manera que sea el modelo el que se ajuste a éstos. Para ello, es necesario la importación de las siguientes librerías de SkLearn:

In [48]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from  sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
import time

### 4.1. División de los datos: particiones de entrenamiento y test

Para establecer la división de los datos haremos uso de la función _train_test_split()_,  en la que especificaremos la metodología __Holdout__ que divide las particiones de entrenamiento y test en una proporción (2/3, 1/3).

También inicializaremos el atributo _random_state_ con un valor predefinido, ya que esto garantiza que se mantenga el mismo conjunto de datos de partición de entrenamiento y de test en todas las ejecuciones. En el caso de inicializarlo a _None_, estaríamos dejando al azar el entrenamiento del modelo, ya que hay algunas métricas como en KNN que no se basan en estadísticas globales de los datos, y esto puede variar el resultado cada vez que ejecutemos el código.

El valor elegido para _random_state_ será _42_ (número aleatorio).

Una vez especificado esto, se procede a la partición de los datos.

In [49]:
X = datos.drop("Attrition", axis=1)
y = datos["Attrition"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

### 4.2. Escalado e imputación de la partición de entrenamiento.

Una vez divididos los datos, buscaremos el modelo que mejor se adapte a la partición de entrenamiento según el tipo de imputación y escalado de datos. Para los escaladores usaremos los tres escaladores estudiados en clase para KNN: __MinMax (to 0-1 range), Estandarización y RobustScaler__; mientras que para los imputadores usaremos los especificados en el enunciado: __la media y la mediana__ de las variables de la misma instancia (imputación univariante).

También usaremos como clasificador KNN y como métrica para la puntuación de cada imputer/scaler __balanced accuracy__:

In [50]:
# Lista de escaladores e imputadores
Scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
Imputers = [SimpleImputer(strategy="mean"), SimpleImputer(strategy="median")]

# Variables para almacenar el mejor modelo y su puntuación
mejor_scaler = None
mejor_imputer = None
mejor_score = 0

# Defiinimos el transformer para columnas categóricas
categorical_transformer = Pipeline(steps=[
  ("encoder", OneHotEncoder(handle_unknown="ignore"))
  ])

# Iteramos sobre los escaladores
for scaler in Scalers:

  # Iteramos sobre los imputadores
  for imputer in Imputers:

    # Pipeline para las columnas numéricas
    numerical_transformer = Pipeline(steps=[
      ("imputer", imputer), ("scaler", scaler)
      ])
    
    # Preprocesador combinado
    preprocessor = ColumnTransformer(transformers=[
      ("num", numerical_transformer, numericas), 
      ("cat", categorical_transformer, categoricas)
      ])
    
    # Pipeline completo con clasificador (se utiliza KNN para clasificar)
    pipeline = Pipeline(steps=[
      ("preprocessor", preprocessor), 
      ("classifier", KNeighborsClassifier())
      ])
    
    # Evaluación del modelo con 3-fold cross-validation (se utiliza balanced_accuracy como métrica)
    score = cross_val_score(pipeline, X_train, y_train, 
                            cv=3, scoring="balanced_accuracy").mean()
    
    # Comparamos con el mejor modelo
    if score > mejor_score:
      mejor_score = score
      mejor_scaler = scaler
      mejor_imputer = imputer
    
    # Imprimimos los resultados de cada combinación
    print(f"Scaler: {scaler}, Imputer: {imputer}, Score: {score}")

# Obtenemos el mejor scaler/imputer y su score
print(f"\nMejor Scaler: {mejor_scaler}, Mejor Imputer: {mejor_imputer}, Mejor Score: {mejor_score}")

Scaler: StandardScaler(), Imputer: SimpleImputer(), Score: 0.5761237407807159
Scaler: StandardScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.5764301133297356
Scaler: MinMaxScaler(), Imputer: SimpleImputer(), Score: 0.573666014570966
Scaler: MinMaxScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.573666014570966
Scaler: RobustScaler(), Imputer: SimpleImputer(), Score: 0.5883600917431192
Scaler: RobustScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.5834665632307968

Mejor Scaler: RobustScaler(), Mejor Imputer: SimpleImputer(), Mejor Score: 0.5883600917431192


Observando los datos podemos concluir que el mejor método de escalado es __RobustScaler()__, el cual es particularmente útil cuando existen valores atípicos en los datos, ya que transforma los atributos usando la mediana y el rango intercuartílico; mientras que el mejor método de imputación (univariante) es __SimpleImputer(strategy="mean")__, el cual reemplaza los valores faltantes con la media de cada atributo. Esta estrategia es muy común y sobretodo adecuada cuando los datos son aproximadamente simétricos.

### 4.3. Modelos iniciales con hiperparámetros por defecto

Una vez tenemos los métodos de escalado y de imputación seleccionados podemos pasar a la siguiente fase. Antes de realizar el ajuste de hiperparámetros, primeramente buscaremos los modelos de KNN y Árboles de Decisión con los hiperparámetros por defecto.

In [ ]:
# Definimos los clasificadores KNN y DecisionTreeClassifier
clasificadores = [KNeighborsClassifier(), DecisionTreeClassifier()]

# Definimos el codificador de variables categóricas
categorical_transformer = Pipeline(steps=[
       ("encoder", OneHotEncoder(handle_unknown="ignore"))
       ])

# Definimos el mejor escalador e imputador obtenidos anteriormente
numerical_transformer = Pipeline(steps=[
       ("imputer", mejor_imputer), 
       ("scaler", mejor_scaler)
       ])

# Preprocesador combinado para columnas numéricas y categóricas
preprocessor = ColumnTransformer(transformers=[
       ("num", numerical_transformer, numericas), 
       ("cat", categorical_transformer, categoricas)
       ])

# Iteramos sobre los clasificadores
for clasificador in clasificadores:

    # Pipeline completo con clasificador
    pipeline = Pipeline(steps=[
           ("preprocessor", preprocessor), 
           ("classifier", clasificador)
           ])
    
    # Definimos los hiperparámetros por defecto para GridSearch
    if isinstance(clasificador, KNeighborsClassifier):
            param_grid = {
                'classifier__n_neighbors': [5],
                'classifier__weights': ['uniform'],
                'classifier__metric': ['minkowski']
            }
    elif isinstance(clasificador, DecisionTreeClassifier):
            param_grid = {
                'classifier__criterion': ['gini'],
                'classifier__max_depth': [None],
                'classifier__min_samples_split': [2],
                'classifier__min_impurity_decrease': [0.0]
            }
    
    # Usamos GridSearchCV para encontrar los mejores hiperparámetros
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy')
    
    # Medimos el tiempo de entrenamiento del modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor puntuación:", grid_search.best_score_)
    print("Tiempo de ejecución:", end_time - start_time, "segundos")
# Mostrar los mejores hiperparámetros

Mejores parámetros: {'classifier__metric': 'minkowski', 'classifier__n_neighbors': 5, 'classifier__weights': 'uniform'}
Mejor puntuación: 0.5883600917431192
Tiempo de ejecución: 0.06190776824951172 segundos
Mejores parámetros: {'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_split': 2}
Mejor puntuación: 0.7611126101816873
Tiempo de ejecución: 0.0910196304321289 segundos


In [ ]:
# Creamos los clasificadores principales (KNN y Árboles de decisión)
clasificadores = [KNeighborsClassifier(), DecisionTreeClassifier()]

# Definimos el codificador de variables categóricas
categorical_transformer = Pipeline(steps=[
       ("encoder", OneHotEncoder(handle_unknown="ignore"))
       ])

# Definimos el mejor escalador e imputador obtenidos anteriormente en una pipeline
numerical_transformer = Pipeline(steps=[
       ("imputer", mejor_imputer), 
       ("scaler", mejor_scaler)
       ])

# Preprocesador combinado para columnas numéricas y categóricas
preprocessor = ColumnTransformer(transformers=[
       ("num", numerical_transformer, numericas), 
       ("cat", categorical_transformer, categoricas)
       ])

# Iteramos sobre los clasificadores
for clasificador in clasificadores:

    # Definimos una pipeline con el preprocesador y el clasificador
    pipeline = Pipeline(steps=[
           ("preprocessor", preprocessor), 
           ("classifier", clasificador)
           ])
    
    # Definimos la rejilla de hiperparámetros para cada clasificador
    if isinstance(clasificador, KNeighborsClassifier):
            param_grid = {
                'classifier__n_neighbors': [3, 5, 7, 9, 11],
                'classifier__weights': ['uniform', 'distance'],
                'classifier__metric': ['minkowski', 'euclidean', 'manhattan']
            }
    elif isinstance(clasificador, DecisionTreeClassifier):
            param_grid = {
                'classifier__criterion': ['gini', 'entropy'],
                'classifier__max_depth': [5, 10, 15, None],
                'classifier__min_samples_split': [2, 5, 10],
                'classifier__min_impurity_decrease': [0.0, 0.01, 0.02]
            }
            
    # Entrenamos el modelo con GridSearchCV para encontrar los mejores hiperparámetros
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy')
    
    # Medimos el tiempo del entrenamiento del modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    # Mostrar los mejores hiperparámetros
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor puntuación:", grid_search.best_score_)
    print("Tiempo de ejecución:", end_time - start_time, "segundos")

Mejores parámetros: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 5, 'classifier__weights': 'distance'}
Mejor puntuación: 0.8067087155963303
Tiempo de ejecución: 0.8216135501861572 segundos
Mejores parámetros: {'classifier__criterion': 'entropy', 'classifier__max_depth': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_split': 2}
Mejor puntuación: 0.7724180383162439
Tiempo de ejecución: 1.4485673904418945 segundos


Vamos a comprobar los resultados obtenidos con el formato dummy

In [53]:
y_train = datos["Attrition"].iloc[X_train.index]  # Assuming X_train.index holds the original indices
y_test = datos["Attrition"].iloc[X_test.index]  # Assuming X_test.index holds the original indices

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", DummyClassifier())])
param_grid = {
    'classifier__strategy': ['most_frequent', 'stratified', 'prior', 'uniform',  # Removed 'constant'
                            'constant'],  # Added 'constant' back with a constant value
    'classifier__constant': [0, 1] if 'constant' in ['most_frequent', 'stratified', 'prior', 'uniform', 'constant'] else [] # Specify constant values only when 'constant' strategy is used
}
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # Encode y_train for the DummyClassifier
y_test_encoded = label_encoder.transform(y_test)
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy', error_score='raise')
start_time = time.time()
grid_search.fit(X_train, y_train_encoded)
dummy_pred = grid_search.predict(X_test)
end_time = time.time()

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación:", grid_search.best_score_)
print("Tiempo de ejecución:", end_time - start_time, "segundos")

Mejores parámetros: {'classifier__constant': 1, 'classifier__strategy': 'uniform'}
Mejor puntuación: 0.5223320291419321
Tiempo de ejecución: 0.28276777267456055 segundos


# Avanzados: Modelos lineales y SVM's